In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv', error_bad_lines=False)
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
data.isna().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [4]:
scaler = MinMaxScaler()
df = data.drop(columns=['date','lights'])
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X_df = df.drop(columns=['Appliances'])
Y_df = df['Appliances']

In [5]:
x = np.array(X_df['T2']).reshape(-1,1)
y = X_df['T6']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
pred_y = linear_model.predict(x_test)

print('r-square value:',round(r2_score(y_test, pred_y),2))

r-square value: 0.64


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X_df, Y_df, test_size=0.3, random_state=42)

linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
pred_y = linear_model.predict(x_test)

print('mean absolute error:',round(mean_absolute_error(y_test, pred_y), 2))

mean absolute error: 0.05


In [7]:
print('Residual Sum of Squares:',round(np.sum(np.square(y_test - pred_y)), 2))

Residual Sum of Squares: 45.35


In [8]:
print('Root Mean Squared Error:',round(np.sqrt(mean_squared_error(y_test, pred_y)), 3))

Root Mean Squared Error: 0.088


In [9]:
print('Coefficient of Determination:',round(r2_score(y_test, pred_y),2))

Coefficient of Determination: 0.15


In [10]:
def get_weights(model, feat):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', 'Weight']
    weights_df['Weight'].round(3)
    return weights_df

In [11]:
linear_model_weights = get_weights(linear_model, x_train)

linear_model_weights

,Features,Weight
0,rv2,-3.332941e+10
1,RH_2,-4.567047e-01
2,T_out,-3.218499e-01
3,T2,-2.361828e-01
4,T9,-1.899313e-01
5,RH_8,-1.576016e-01
6,RH_out,-7.766316e-02
7,RH_7,-4.460958e-02
8,RH_9,-3.980246e-02
9,T5,-1.565270e-02


In [12]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

pred_y = ridge_reg.predict(x_test)

print('root mean squared error:',round(np.sqrt(mean_squared_error(y_test, pred_y)), 3))

root mean squared error: 0.088


In [13]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

lasso_model_weights = get_weights(lasso_reg, x_train)
lasso_model_weights

,Features,Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [14]:
pred_y = lasso_reg.predict(x_test)

print('root mean squared error:',round(np.sqrt(mean_squared_error(y_test, pred_y)), 3))

root mean squared error: 0.094
